In [1]:
%matplotlib inline
import os
from skimage.transform import resize
from skimage.io import imread, imsave
import random
from six.moves import cPickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lasagne.nonlinearities import softmax
from lasagne import layers, nonlinearities
from nolearn.lasagne import NeuralNet, BatchIterator
path = os.getcwd()

/home/sruti/anaconda3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/home/sruti/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Function to read dataset
def read_data(typeData, labelsInfo, imageSize, path):
    x = np.zeros((labelsInfo.shape[0], imageSize))
    
    for (index, idImage) in enumerate(labelsInfo['ID']):
        nameFile = '{0}/{1}Resized32/{2}.Bmp'.format(path, typeData, idImage)
        img = imread(nameFile, as_grey = True)
        x[index, :] = np.reshape(img, (1, imageSize))   
    return x

#Convolutional Neural Network Model
def fit_model(reshaped_train_x, y, image_width, image_height, reshaped_test_x):
    net = NeuralNet(
        layers = [
            ('input', layers.InputLayer),
            ('conv1', layers.Conv2DLayer),
            ('pool1', layers.MaxPool2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('conv2', layers.Conv2DLayer),
            ('pool2', layers.MaxPool2DLayer),
            ('dropout2', layers.DropoutLayer),
            ('conv3', layers.Conv2DLayer),
            ('hidden4', layers.DenseLayer),
            ('output', layers.DenseLayer),
        ],
        input_shape = (None, 1, 32, 32),
        conv1_num_filters=32, conv1_filter_size=(5, 5), 
        pool1_pool_size=(2, 2),
        dropout1_p=0.2,
        conv2_num_filters=64, conv2_filter_size=(5, 5), 
        pool2_pool_size=(2, 2),
        dropout2_p=0.2,
        conv3_num_filters = 128, conv3_filter_size = (5, 5),
        hidden4_num_units=500,
        output_num_units = 62, output_nonlinearity = softmax,
        
        update_learning_rate = 0.01,
        update_momentum = 0.9,
        
        batch_iterator_train = BatchIterator(batch_size = 100),
        batch_iterator_test = BatchIterator(batch_size = 100),
        
        use_label_encoder = True,
        regression = False,
        max_epochs = 100,
        verbose = 1,
    )
    
    net.fit(reshaped_train_x, y)
    prediction = net.predict(reshaped_test_x)
    
    return prediction

In [3]:
from numpy import array
imageSize = 1024 
image_width = image_height = int(imageSize ** 0.5)

labelsInfoTrain = pd.read_csv('{0}/trainLabels.csv'.format(path))
labelsInfoTest = pd.read_csv('{0}/sampleSubmission.csv'.format(path))

# Load dataset
xTrain = read_data('train', labelsInfoTrain, imageSize, path)
xTest = read_data('test', labelsInfoTest, imageSize, path)

yTrain = list(map(ord, labelsInfoTrain['Class']))
yTrain = np.array(yTrain)

In [4]:
xTrain /= xTrain.std(axis = None)
xTrain -= xTrain.mean()

xTest /= xTest.std(axis = None)
xTest -= xTest.mean()

In [5]:
train_x_reshaped = xTrain.reshape(xTrain.shape[0], 1, image_height, image_width).astype('float32')

test_x_reshaped = xTest.reshape(xTest.shape[0], 1, image_height, image_width).astype('float32')

In [6]:
#Make Prediction
predict = fit_model(train_x_reshaped, yTrain, image_width, image_height, test_x_reshaped)

/home/sruti/anaconda3/lib/python3.5/site-packages/lasagne/layers/conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)
/home/sruti/anaconda3/lib/python3.5/site-packages/lasagne/layers/conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)


# Neural Network with 352586 learnable parameters

## Layer information

  #  name      size
---  --------  --------
  0  input     1x32x32
  1  conv1     32x28x28
  2  pool1     32x14x14
  3  dropout1  32x14x14
  4  conv2     64x10x10
  5  pool2     64x5x5
  6  dropout2  64x5x5
  7  conv3     128x1x1
  8  hidden4   500
  9  output    62

  epoch    trn loss    val loss    trn/val    valid acc  dur
-------  ----------  ----------  ---------  -----------  ------
      1     4.08684     4.02057    1.01648      0.07254  29.35s
      2     3.89902     3.83489    1.01672      0.04836  28.98s
      3     3.80136     3.80941    0.99789      0.04914  29.28s
      4     3.78883     3.79315    0.99886      0.04914  30.51s
      5     3.78095     3.78858    0.99799      0.07176  31.31s
      6     3.77807     3.78690    0.99767      0.07254  27.72s
      7     3.77620     3.78618    0.99736      0.07254  27.75s
      8     3.77502     3.78605    0.99709      0.07254  30.98s
      9     3.77410   

In [7]:
#Writing to submission file
yTest = list(map(chr, predict))
labelsInfoTest['Class'] = yTest
labelsInfoTest.to_csv('{0}/subzero.csv'.format(path), index = False)